In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [2]:
# Define paths
real_videos_path = r"C:\Users\acer\DeepFake\Dataset\archive_data\FAKE"
fake_videos_path = r"C:\Users\acer\DeepFake\Dataset\archive_data\REAL"
output_frames_dir = r"C:\Users\acer\DeepFake\Dataset\archive_data\ex_frame"

# Ensure frame directories exist
real_frames_dir = os.path.join(output_frames_dir, "real")
fake_frames_dir = os.path.join(output_frames_dir, "fake")
os.makedirs(real_frames_dir, exist_ok=True)
os.makedirs(fake_frames_dir, exist_ok=True)

# Parameters
frame_size = 128  # Resize all frames to 128x128
frames_per_video = 20  # Number of frames to sample from each video


In [3]:
def extract_and_save_frames(video_path, save_dir, frame_size, frames_per_video):
    """
    Extract frames from a video and save them as images.
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_interval = max(1, total_frames // frames_per_video)
    
    for i in range(frames_per_video):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * sample_interval)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (frame_size, frame_size))
            frame_filename = f"{os.path.basename(video_path)}_frame_{i}.jpg"
            cv2.imwrite(os.path.join(save_dir, frame_filename), frame)
    cap.release()

In [4]:
# Process videos and extract frames
for video_file in os.listdir(real_videos_path):
    extract_and_save_frames(os.path.join(real_videos_path, video_file), real_frames_dir, frame_size, frames_per_video)

for video_file in os.listdir(fake_videos_path):
    extract_and_save_frames(os.path.join(fake_videos_path, video_file), fake_frames_dir, frame_size, frames_per_video)



In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

In [6]:
# Train and Validation Generators
train_generator = datagen.flow_from_directory(
    output_frames_dir,
    target_size=(frame_size, frame_size),
    batch_size=32,
    class_mode='categorical',  # Keep categorical
    subset="training"
)

val_generator = datagen.flow_from_directory(
    output_frames_dir,
    target_size=(frame_size, frame_size),
    batch_size=32,
    class_mode='categorical',  # Keep categorical
    subset="validation"
)



Found 13632 images belonging to 2 classes.
Found 3408 images belonging to 2 classes.


In [7]:
# Define Model

base_model = Xception(weights='imagenet', include_top=False, input_shape=(frame_size, frame_size, 3))
base_model.trainable = False  # Freeze the base model to speed up training

model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ (None, 4, 4, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,056,170 (95.58 MB)

 Trainable params: 4,194,690 (16.00 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [8]:
# Train Model
model.fit(train_generator, validation_data=val_generator, epochs=10, steps_per_epoch=100,)
 

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 398s 4s/step - accuracy: 0.5262 - loss: 1.2121 - val_accuracy: 0.5062 - val_loss: 0.6931
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 367s 4s/step - accuracy: 0.5126 - loss: 0.6941 - val_accuracy: 0.5062 - val_loss: 0.6931
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 363s 4s/step - accuracy: 0.5040 - loss: 0.6958 - val_accuracy: 0.5062 - val_loss: 0.6933
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 380s 4s/step - accuracy: 0.4996 - loss: 0.6941 - val_accuracy: 0.5062 - val_loss: 0.6931
Epoch 5/10
 26/100 ━━━━━━━━━━━━━━━━━━━━ 2:12 2s/step - accuracy: 0.4691 - loss: 0.6934

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


100/100 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.4973 - loss: 0.6930 - val_accuracy: 0.5065 - val_loss: 0.6931
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 361s 4s/step - accuracy: 0.4977 - loss: 0.6933 - val_accuracy: 0.5062 - val_loss: 0.6931
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 358s 4s/step - accuracy: 0.5045 - loss: 0.6930 - val_accuracy: 0.5062 - val_loss: 0.6931
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 252s 3s/step - accuracy: 0.5039 - loss: 0.6931 - val_accuracy: 0.5062 - val_loss: 0.6931
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 228s 2s/step - accuracy: 0.5189 - loss: 0.6931 - val_accuracy: 0.5062 - val_loss: 0.6931
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.4836 - loss: 0.6934 - val_accuracy: 0.5062 - val_loss: 0.6931


In [14]:
# Save the model

model.save('My_model3.keras')

In [10]:
test_generator = datagen.flow_from_directory(
    output_frames_dir,
    target_size=(frame_size, frame_size),
    batch_size=32,
    class_mode='categorical'  # Keep categorical
)


Found 17040 images belonging to 2 classes.


In [11]:
# Evaluate on test data
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


533/533 ━━━━━━━━━━━━━━━━━━━━ 555s 1s/step - accuracy: 0.5052 - loss: 0.6931
Test Accuracy: 50.60%


In [12]:
# Plot training history
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


NameError: name 'history' is not defined

In [17]:
import numpy as np
import cv2
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.models import load_model

def extract_frames(video_path, frame_size, frames_per_video):
    """
    Extract frames from a video for deepfake detection.
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_interval = max(1, total_frames // frames_per_video)
    frames = []

    for i in range(frames_per_video):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * sample_interval)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (frame_size, frame_size))
            frame = frame.astype('float32') / 255.0  # Normalize
            frames.append(frame)

    cap.release()
    
    if len(frames) == 0:
        print("Error: No frames extracted from video.")
        return None
    
    return np.array(frames)

def detect_deepfake(video_path, model, frame_size=128, frames_per_video=20):
    """
    Detects whether a video is real or deepfake using the trained model.
    """
    frames = extract_frames(video_path, frame_size, frames_per_video)
    
    if frames is None or len(frames) == 0:
        return None, None  # Return if no frames were extracted

    frames = np.expand_dims(frames, axis=-1)  # Ensure correct shape
    predictions = model.predict(frames)
    average_prediction = np.mean(predictions, axis=0)
    
    is_fake = average_prediction[1] > 0.5
    confidence = average_prediction[1] if is_fake else average_prediction[0]

    return is_fake, confidence

# Load trained model
model_path = r"C:\Users\acer\DeepFake\backend\My_model3.keras"
model = load_model(model_path)

# Test detection on a sample video
video_path = r"C:\Users\acer\DeepFake\Dataset\archive_data\FAKE\id7_id11_0000.mp4"
is_fake, confidence = detect_deepfake(video_path, model)

if is_fake is not None:
    print(f"Video is {'FAKE' if is_fake else 'REAL'} with {confidence * 100:.2f}% confidence.")
else:
    print("Failed to analyze video.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Video is REAL with 50.25% confidence.
